In [87]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, datetime, os
from pathlib import Path

## Create folder with today's date
today = datetime.date.today()  
todaystr = today.isoformat()   
dir_path = os.getcwd()
save_dir = dir_path + "/" + todaystr
os.mkdir(save_dir)

## Set the target save location and Firefox preferences to auto download csv files
profile = webdriver.FirefoxProfile()
profile.set_preference("browser.download.folderList", 2)
profile.set_preference("browser.download.manager.showWhenStarting", False)
profile.set_preference("browser.download.dir", save_dir)
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "text/csv")

## Create the Firefox session
driver = webdriver.Firefox(firefox_profile=profile)



In [88]:
## Get past the disclaimer splash screen
driver.get("http://casesearch.courts.state.md.us/casesearch/inquiry-index.jsp")
check = driver.find_element_by_name("disclaimer")
check.click()
continue_button = driver.find_element_by_name("action")
continue_button.click()

assert "No results found." not in driver.page_source

In [89]:
## The search page won't let you search for all cases, need to do 26 searches (A-Z)

## Enter search criteria
last_name = driver.find_element_by_name("lastName")
last_name.clear()
last_name.send_keys("E") ## Turn to variable
driver.find_element_by_xpath("//select[@name='partyType']/option[text()='Plaintiff']").click()
driver.find_element_by_xpath("//input[@name='site' and @value='CIVIL']").click()
driver.find_element_by_xpath("//input[@name='courtSystem' and @value='D']").click()
driver.find_element_by_xpath("//select[@name='countyName']/option[text()='Charles County']").click()
last_name = driver.find_element_by_name("filingStart")
last_name.clear()
last_name.send_keys("6/1/2019")  ## Turn to variable
last_name = driver.find_element_by_name("filingEnd")
last_name.clear()
last_name.send_keys("9/30/2019")  ## Turn to variable
driver.find_element_by_xpath("//input[@name='action' and @value='Search']").click()

element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.LINK_TEXT, "CSV"))).click()


In [62]:
driver.find_element_by_link_text('CSV').click()

In [24]:
driver.close()